In [47]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}

In [48]:
-- import Prelude hiding ((++))
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA

import qualified Data.Map.Strict as M
import Data.Map.Strict ((!))

import Control.Monad (when)
import Control.Monad.State.Lazy
import Control.Monad.Reader
import Control.Monad.Writer

In [49]:
data Location = Literal Integer | Register Char deriving (Show, Eq)
data Instruction =   Snd Location
                   | Set Location Location 
                   | Add Location Location 
                   | Mul Location Location
                   | Mod Location Location
                   | Rcv Location
                   | Jgz Location Location
                   deriving (Show, Eq)

data Machine = Machine { registers :: M.Map Char Integer
                       , lastSound :: Integer
                       , pc :: Int
                       } 
               deriving (Show, Eq)

type ProgrammedMachine = WriterT [Integer] (ReaderT [Instruction] (State Machine)) ()

In [50]:
emptyMachine = Machine {registers = M.empty, lastSound = 0, pc = 0}

In [51]:
sc :: Parser ()
sc = L.space (skipSome spaceChar) CA.empty CA.empty

lexeme  = L.lexeme sc

integer       = lexeme L.integer
signedInteger = L.signed sc integer

symb = L.symbol sc

-- reg :: Parser String
-- reg = id <$> some letterChar

reg = lexeme (some letterChar)

location = (Literal <$> signedInteger) <|> register
register = (Register . head) <$> reg

instructionsP = instructionP `sepBy` space
instructionP = choice [sndP, setP, addP, mulP, modP, rcvP, jgzP]

sndP = Snd <$> (try (symb "snd") *> location)
setP = Set <$> (try (symb "set") *> register) <*> location
addP = Add <$> (try (symb "add") *> register) <*> location
mulP = Mul <$> (try (symb "mul") *> register) <*> location
modP = Mod <$> (try (symb "mod") *> register) <*> location
rcvP = Rcv <$> (try (symb "rcv") *> location)
jgzP = Jgz <$> (try (symb "jgz") *> location) <*> location

successfulParse :: Text -> [Instruction]
successfulParse input = 
        case parse instructionsP "input" input of
                Left  _error -> [] -- TIO.putStr $ T.pack $ parseErrorPretty err
                Right instructions  -> instructions

In [52]:
sample = T.pack "set a 1\nadd a 2\nmul a a\nmod a 5\nsnd a\nset a 0\nrcv a\njgz a -1\nset a 1\njgz a -2"

In [53]:
successfulParse sample

[Set (Register 'a') (Literal 1),Add (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Mod (Register 'a') (Literal 5),Snd (Register 'a'),Set (Register 'a') (Literal 0),Rcv (Register 'a'),Jgz (Register 'a') (Literal (-1)),Set (Register 'a') (Literal 1),Jgz (Register 'a') (Literal (-2))]

In [54]:
sampleInstructions = successfulParse sample
sampleInstructions

[Set (Register 'a') (Literal 1),Add (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Mod (Register 'a') (Literal 5),Snd (Register 'a'),Set (Register 'a') (Literal 0),Rcv (Register 'a'),Jgz (Register 'a') (Literal (-1)),Set (Register 'a') (Literal 1),Jgz (Register 'a') (Literal (-2))]

In [55]:
evaluate :: Machine -> Location -> Integer
evaluate _ (Literal i)  = i
evaluate m (Register r) = M.findWithDefault 0 r (registers m)

In [56]:
applyInstruction :: Instruction -> Machine -> Machine

applyInstruction (Snd sound) m = m {lastSound = freq, pc = pc'}
    where pc' = pc m + 1
          freq = evaluate m sound

applyInstruction (Set (Register a) b) m = m {registers = reg', pc = pc'}
    where pc' = pc m + 1
          y = evaluate m b
          reg' = M.insert a y $ registers m

applyInstruction (Add (Register a) b) m = m {registers = reg', pc = pc'}
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x + y) $ registers m

applyInstruction (Mul (Register a) b) m = m {registers = reg', pc = pc'}
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x * y) $ registers m

applyInstruction (Mod (Register a) b) m = m {registers = reg', pc = pc'}
    where pc' = pc m + 1
          x = evaluate m (Register a) 
          y = evaluate m b
          reg' = M.insert a (x `mod` y) $ registers m

applyInstruction (Rcv a) m = m {pc = pc'}
    where pc' = pc m + 1
    
applyInstruction (Jgz a b) m = m {pc = pc'}
    where x = evaluate m a
          y = evaluate m b
          pc' = if x > 0 then pc m + (fromIntegral y) else pc m + 1

In [57]:
executeInstruction :: ProgrammedMachine
executeInstruction =
    do  instrs <- ask
        m <- get
        let instr = instrs!!(pc m)
--         tell [("pc = " ++ show (pc m))]
        put (applyInstruction instr m)

In [58]:
isRecover :: Instruction -> Bool
isRecover (Rcv _) = True
isRecover _ = False

In [59]:
-- handleRecover :: ProgrammedMachine
-- handleRecover = 
--     do  instrs <- ask
--         m <- get
--         let instr = instrs!!(pc m)
--         when (isReceive instr)
--             $
--             do let Rcv a = instr
--                let x = evaluate m a
--                when (x /= 0) (tell (["reccovering " ++ (show (lastSound m))]))

In [60]:
recoverTriggers :: [Instruction] -> Machine -> Bool
recoverTriggers instrs m = 
        if isRecover instr
        then (x /= 0)
        else False
        where instr = instrs!!(pc m)
              Rcv a = instr
              x = evaluate m a

In [61]:
executeInstructions = 
    do  instrs <- ask
        m <- get
--         tell ["instrs = " ++ (show instrs)]
        when (pc m >= 0 && pc m < length instrs)
            $
            do let rt = recoverTriggers instrs m
               if rt
               then tell [lastSound m]
               else do executeInstruction
                       executeInstructions

In [62]:
runState (runReaderT (runWriterT executeInstructions) sampleInstructions ) emptyMachine

(((),[4]),Machine {registers = fromList [('a',1)], lastSound = 4, pc = 6})

In [63]:
runState (
    runReaderT (
        runWriterT executeInstructions
               ) 
        (take 7 sampleInstructions) 
         ) 
         emptyMachine

(((),[]),Machine {registers = fromList [('a',0)], lastSound = 4, pc = 7})

In [64]:
sampleInstructions

[Set (Register 'a') (Literal 1),Add (Register 'a') (Literal 2),Mul (Register 'a') (Register 'a'),Mod (Register 'a') (Literal 5),Snd (Register 'a'),Set (Register 'a') (Literal 0),Rcv (Register 'a'),Jgz (Register 'a') (Literal (-1)),Set (Register 'a') (Literal 1),Jgz (Register 'a') (Literal (-2))]

In [65]:
part1 instructions = 
    runState (
        runReaderT (
            runWriterT executeInstructions
                   ) 
            instructions 
             ) 
             emptyMachine

In [68]:
main :: IO ()
main = do 
        text <- TIO.readFile "../../data/advent18.txt"
        let instrs = successfulParse text
        let ((result, l), machinef) = part1 instrs
        print $ head l
--         print $ part2 instrs

In [69]:
main

1187